In [ ]:
# periodic

import numpy as np
import matplotlib.pyplot as plt
import sys
import time
import os




In [ ]:


cwd = os.getcwd()   # the current directory
print(cwd)




In [ ]:
#generate training data
n_sample = 3000
example_time_steps = 6
n_time_steps = 300
n_time_skip = 6


Nx = 200
N1 = Nx

L = 1.
c = .1
T = 1.23
K = 30


dx = L/Nx
dt = 0.8*dx/c

x = np.arange(Nx)*dx
NNc = 11


c2 = c**2.


def init(s):
    if s <= 1:
        return 1.-1.5*s**2.+0.75*s**3.
    elif 1<s<2:

        return 0.25*(2.-s)**3.
    return 0.


print(f'final time is {dt*n_time_steps}')
print(f'simulation time step is {dt}')
print(f'flow map time step is {dt*n_time_skip}')


Z = np.array([])

# loop for sample


vinit = np.vectorize(init)

np.random.seed(123)
x0 = 0.5 + np.random.uniform(-0.2,0.2,size=n_sample)
a0 = np.random.uniform(7.,9.,size=n_sample)

z_list = []
p_list = []
q_list = []

for x0_i, a0_i in zip(x0, a0):
    q = vinit(a0_i*np.abs(x-x0_i))
    p = np.zeros(N1, dtype=np.float64)
    z = np.concatenate([q, p], axis=0)

    z_list.append(z)
    p_list.append(p)
    q_list.append(q)

z = np.array(z_list) # convert list to numpy ndarray
p = np.array(p_list) # convert list to numpy ndarray
q = np.array(q_list) # convert list to numpy ndarray

# Z = z.copy()[np.newaxis,...]
Z_list = []
Z_list.append(z)

#print(p.shape)

In [ ]:
for i in range(1,n_time_steps):


    q0 = q + 0.5*dt*p

    p = p + c2*dt*(np.roll(q0,-1,axis=1) - 2.*q0 + np.roll(q0,1,axis=1))/dx**2.

    q = q0 + 0.5*dt*p;

    # scale p
    # p = p/(np.pi*Nc)

    # qe = np.zeros_like(x)
    # pe = np.zeros_like(x)
    # TT = dt*(i+1.)
    # for n in range(Nc):
    #     qe = qe + np.sin((n+1)*np.pi*x)*(a[n]*np.cos(c*(n+1)*np.pi*TT) + b[n]/(np.pi*(n+1))*np.sin(c*(n+1)*np.pi*TT))
    #     pe = pe + np.sin((n+1)*np.pi*x)*(-c*a[n]*(n+1)*np.pi*np.sin(c*(n+1)*np.pi*TT) + c*b[n]*np.cos(c*(n+1)*np.pi*TT))

    # z = np.vstack([z, np.concatenate([qe, pe])])
    z = np.concatenate([q, p], axis=1)

    #ham[i] = 0.5*dx*np.dot(p,p) + c2*0.5/dx*(np.dot(q-np.roll(q,1),q-np.roll(q,1)))

    # Z = np.concatenate((Z, z[np.newaxis,...]))
    Z_list.append(z)

   

Z = np.array(Z_list)

print(Z.shape)



# Z = Z[::n_time_skip,:,:]
Z = Z[::n_time_skip]
n_time_steps = Z.shape[0]


output_list = []
for shift in range(1, example_time_steps+1):
# output time series, starting from each single time step

    output_slice = Z[shift: n_time_steps - example_time_steps + shift]
# merge time and sample dimension as required by training
    n_time, _, n_x = output_slice.shape
    output_slice = np.reshape(output_slice, [n_sample * n_time, n_x])
    output_list.append(output_slice)

labels = np.stack(output_list, axis=1)

features = Z[:-example_time_steps,:,:]
features = features.reshape([n_time*n_sample,N1*2])





# # the shape of input training data is (n_time, n_sample, n_x) or (n_time * n_sample, n_x)

data_features = f"Wave_bump_peri_features_c_{c}_Nx_{N1}_sample_{n_sample}_timesteps_{n_time}_dt_{dt*n_time_skip*n_time_steps}"
data_labels = f"Wave_bump_peri_labels_c_{c}_Nx_{N1}_sample_{n_sample}_timesteps_{n_time}_dt_{dt*n_time_skip*n_time_steps}"

# ensure the data directory exists
data_dir = os.path.join(cwd, 'data')
os.makedirs(data_dir, exist_ok=True)  # create the data directory if it doesn't exist

features_file = os.path.join(cwd, 'data', data_features + ".npy")
labels_file = os.path.join(cwd,  'data', data_labels + ".npy")





In [ ]:
# train features and labels
target_file = np.save(features_file, features)
target_file = np.save(labels_file, labels)

In [ ]:
#generate the test data, data_24
n_sample = 3000
example_time_steps = 6
n_time_steps = 600
n_time_skip = 6


Nx = 200
N1 = Nx

L = 1.
c = .1
T = 1.23
K = 30


dx = L/Nx
dt = 0.8*dx/c

x = np.arange(Nx)*dx
NNc = 11


c2 = c**2.


def init(s):
    if s <= 1:
        return 1.-1.5*s**2.+0.75*s**3.
    elif 1<s<2:

        return 0.25*(2.-s)**3.
    return 0.


print(f'final time is {dt*n_time_steps}')
print(f'simulation time step is {dt}')
print(f'flow map time step is {dt*n_time_skip}')


Z = np.array([])

# loop for sample


vinit = np.vectorize(init)

np.random.seed(123)
x0 = 0.5 + np.random.uniform(-0.2,0.2,size=n_sample)
a0 = np.random.uniform(7.,9.,size=n_sample)

z_list = []
p_list = []
q_list = []

for x0_i, a0_i in zip(x0, a0):
    q = vinit(a0_i*np.abs(x-x0_i))
    p = np.zeros(N1, dtype=np.float64)
    z = np.concatenate([q, p], axis=0)

    z_list.append(z)
    p_list.append(p)
    q_list.append(q)

z = np.array(z_list) # convert list to numpy ndarray
p = np.array(p_list) # convert list to numpy ndarray
q = np.array(q_list) # convert list to numpy ndarray

# Z = z.copy()[np.newaxis,...]
Z_list = []
Z_list.append(z)


for i in range(1,n_time_steps):


    q0 = q + 0.5*dt*p

    p = p + c2*dt*(np.roll(q0,-1,axis=1) - 2.*q0 + np.roll(q0,1,axis=1))/dx**2.

    q = q0 + 0.5*dt*p;

    # scale p
    # p = p/(np.pi*Nc)

    # qe = np.zeros_like(x)
    # pe = np.zeros_like(x)
    # TT = dt*(i+1.)
    # for n in range(Nc):
    #     qe = qe + np.sin((n+1)*np.pi*x)*(a[n]*np.cos(c*(n+1)*np.pi*TT) + b[n]/(np.pi*(n+1))*np.sin(c*(n+1)*np.pi*TT))
    #     pe = pe + np.sin((n+1)*np.pi*x)*(-c*a[n]*(n+1)*np.pi*np.sin(c*(n+1)*np.pi*TT) + c*b[n]*np.cos(c*(n+1)*np.pi*TT))

    # z = np.vstack([z, np.concatenate([qe, pe])])
    z = np.concatenate([q, p], axis=1)

    #ham[i] = 0.5*dx*np.dot(p,p) + c2*0.5/dx*(np.dot(q-np.roll(q,1),q-np.roll(q,1)))

    # Z = np.concatenate((Z, z[np.newaxis,...]))
    Z_list.append(z)

   

Z = np.array(Z_list)

print(Z.shape)



# Z = Z[::n_time_skip,:,:]
Z = Z[::n_time_skip]
test_data_file = os.path.join(cwd,  'data', "data_24.npy")

np.save(test_data_file, Z)